In [3]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns


In [4]:
df=pd.read_csv('all_kindle_review.csv')
df.head()

,Unnamed: 0.1,Unnamed: 0,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,11539,B0033UV8HI,"[8, 10]",3,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600
1,1,5957,B002HJV4DE,"[1, 1]",5,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400
2,2,9146,B002ZG96I4,"[0, 0]",3,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400
3,3,7038,B002QHWOEU,"[1, 3]",3,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400
4,4,1776,B001A06VJ8,"[0, 1]",4,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000


In [5]:
df=df[['reviewText','rating']]

In [6]:
df

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",3
1,Great short read. I didn't want to put it dow...,5
2,I'll start by saying this is the first of four...,3
3,Aggie is Angela Lansbury who carries pocketboo...,3
4,I did not expect this type of book to be in li...,4
...,...,...
11995,Valentine cupid is a vampire- Jena and Ian ano...,4
11996,I have read all seven books in this series. Ap...,5
11997,This book really just wasn't my cuppa. The si...,3
11998,"tried to use it to charge my kindle, it didn't...",1


In [30]:
y=df['rating']
y

0        0
1        1
2        0
3        0
4        1
        ..
11995    1
11996    1
11997    0
11998   -1
11999    0
Name: rating, Length: 12000, dtype: int64

In [8]:
d={1:-1,2:-1,3:0,4:1,5:1}
df['rating'].replace(d,inplace=True)

C:\Users\vatsa\AppData\Local\Temp\ipykernel_29996\3747976494.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['rating'].replace(d,inplace=True)


In [9]:
df

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",0
1,Great short read. I didn't want to put it dow...,1
2,I'll start by saying this is the first of four...,0
3,Aggie is Angela Lansbury who carries pocketboo...,0
4,I did not expect this type of book to be in li...,1
...,...,...
11995,Valentine cupid is a vampire- Jena and Ian ano...,1
11996,I have read all seven books in this series. Ap...,1
11997,This book really just wasn't my cuppa. The si...,0
11998,"tried to use it to charge my kindle, it didn't...",-1


In [25]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from nltk.corpus import stopwords
sw=set(stopwords.words('english'))

In [26]:
corpus=[]
for i in range(0,len(df)):
    sentence=(df.iloc[i,0])
    sentence=sentence.lower()
    sentence=sentence.split()
    newSentence=[lemmatizer.lemmatize(word) for word in sentence if word not in sw]
    newSentence=" ".join(newSentence)
    corpus.append(newSentence)

In [27]:
len(corpus)


12000

Bag of Words:

In [31]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(corpus,y,test_size=0.2,random_state=42)


In [32]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer=CountVectorizer()
x_train=vectorizer.fit_transform(x_train)
x_test=vectorizer.transform(x_test)

In [36]:
x_train=x_train.toarray()
x_test=x_test.toarray()

In [37]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
model=MultinomialNB()
model.fit(x_train,y_train)
y_pred=model.predict(x_test)
print(classification_report(y_true=y_test,y_pred=y_pred))

              precision    recall  f1-score   support

          -1       0.72      0.83      0.77       803
           0       0.42      0.13      0.20       387
           1       0.78      0.87      0.82      1210

    accuracy                           0.74      2400
   macro avg       0.64      0.61      0.60      2400
weighted avg       0.70      0.74      0.71      2400



In [38]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_true=y_test,y_pred=y_pred))

0.73875


In [46]:
def getSentimentAnaysisScore(inputSentence):
    sentence=inputSentence
    sentence=sentence.lower()
    sentence=sentence.split()
    newSentence=[lemmatizer.lemmatize(word) for word in sentence if word not in sw]
    newSentence=" ".join(newSentence)
    newArr=vectorizer.transform([newSentence]).toarray()
    return model.predict(newArr)
    

In [53]:
print(getSentimentAnaysisScore("The story was poorly written and the plot made no sense, I couldn’t finish it."))

[-1]


In [54]:
print(getSentimentAnaysisScore("The book was okay, some parts were interesting, but overall it didn’t leave much of an impression."))

[0]


In [55]:
print(getSentimentAnaysisScore("Amazing storytelling and vivid descriptions, I couldn’t put it down."))

[1]
